<a href="https://colab.research.google.com/github/thedavidemmanuel/transfer_learning_assignment/blob/main/Transfer_Learning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Install and Setup Kaggle
!pip install kaggle

# Upload your 'kaggle.json' file before running this cell
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Cell 2: Download and Unzip Dataset
!kaggle datasets download -d pkdarabi/bone-break-classification-image-dataset
!unzip -q bone-break-classification-image-dataset.zip -d data
!rm -rf bone-break-classification-image-dataset.zip

In [ ]:
# Step 1: Import Libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import os

# Check TensorFlow version and GPU availability
print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")


# Dataset
DATA_DIR = '/content/data/Bone Break Classification'
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2

# Model training
EPOCHS = 30
LEARNING_RATE = 1e-4

# Models
MODELS = ['VGG16', 'ResNet50', 'InceptionV3']

# Output
OUTPUT_DIR = '/content/bone_break_classification_output'
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# Cell 4: Data Preparation
def create_data_generators():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,            # Augmentation technique 1
        width_shift_range=0.2,        # Augmentation technique 2
        height_shift_range=0.2,       # Augmentation technique 3
        shear_range=0.2,              # Augmentation technique 4
        zoom_range=0.2,               # Augmentation technique 5
        horizontal_flip=True,         # Augmentation technique 6
        fill_mode='nearest',
        validation_split=VALIDATION_SPLIT
    )

    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        DATA_DIR,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training',
        seed=42
    )

    validation_generator = train_datagen.flow_from_directory(
        DATA_DIR,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation',
        seed=42
    )

    # For test generator, create a separate test directory if available
    test_generator = test_datagen.flow_from_directory(
        DATA_DIR,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    return train_generator, validation_generator, test_generator


In [ ]:
# Cell 5: Model Creation
def create_model(model_name, num_classes):
    if model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
    elif model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
    elif model_name == 'InceptionV3':
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
    else:
        raise ValueError(f"Unsupported model: {model_name}")

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=outputs)

    # Unfreeze the top layers of the base model
    for layer in base_model.layers[-10:]:
        layer.trainable = True

    # Freeze the rest of the layers
    for layer in base_model.layers[:-10]:
        layer.trainable = False

    return model


In [ ]:
# Cell 6: Training Function
def train_model(model_name):
    train_generator, validation_generator, test_generator = create_data_generators()
    num_classes = train_generator.num_classes

    if num_classes == 0:
        raise ValueError("No images found in the specified directories. Please check your data path.")

    model = create_model(model_name, num_classes)
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss='categorical_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )

    callbacks = [
        ModelCheckpoint(f"{OUTPUT_DIR}/{model_name}_best.keras", save_best_only=True),
        EarlyStopping(patience=10, restore_best_weights=True),
        ReduceLROnPlateau(factor=0.1, patience=5)
    ]

    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // validation_generator.batch_size,
        epochs=EPOCHS,
        callbacks=callbacks
    )

    # Evaluate on test data
    test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_generator)
    y_true = test_generator.classes
    y_pred = np.argmax(model.predict(test_generator), axis=1)
    f1 = f1_score(y_true, y_pred, average='weighted')

    results = {
        'model': model,
        'history': history,
        'test_loss': test_loss,
        'test_accuracy': test_accuracy,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'f1_score': f1
    }

    return results

In [ ]:
# Cell 7: Training Loop
results = {}

for model_name in MODELS:
    print(f"Training {model_name}")
    try:
        model_results = train_model(model_name)
        results[model_name] = model_results
    except ValueError as e:
        print(f"Error training {model_name}: {str(e)}")
        continue

Training VGG16
Found 904 images belonging to 1 classes.
Found 225 images belonging to 1 classes.
Found 1129 images belonging to 1 classes.
Epoch 1/30


KeyboardInterrupt: 